In [1]:
from ogb.graphproppred import GraphPropPredDataset
from tqdm import tqdm
from graphtoolbox import OgbDataLoader, GraphHelper, RandomWalkEmbedder

/home/cstaib/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_name = 'ogbg-molpcba'

In [3]:
ogb_data_loader = OgbDataLoader()
graph_helper = GraphHelper()
random_walk_embedder = RandomWalkEmbedder()

In [4]:
dataset = GraphPropPredDataset(name=dataset_name)
split_idx = dataset.get_idx_split()

features, labels = [], []
for graph_dict, label in tqdm(dataset):
    graph = ogb_data_loader.graph_dict_to_nx_graph(graph_dict)
    graph = graph_helper.edges_to_nodes(graph)
    random_walk_embedder

100%|██████████| 437929/437929 [03:11<00:00, 2281.58it/s]
